# Codenation - Data Science
<pre>Autor: Leonardo Simões</pre>

## Desafio 8 - Descubra quem fez o ENEM 2016 apenas para treino

Neste desafio deverá descobrir quais estudantes estão fazendo a prova apenas para treino.

### Detalhes
O contexto do desafio gira em torno dos resultados do ENEM 2016 (disponíveis no arquivo train.csv). Este arquivo, e apenas ele, deve ser utilizado para todos os desafios. Qualquer dúvida a respeito das colunas, consulte o [Dicionário dos Microdados do Enem 2016](https://s3-us-west-1.amazonaws.com/acceleration-assets-highway/data-science/dicionario-de-dados.zip).

Alguns estudantes decidem realizar prova do ENEM de forma precoce, como um teste (coluna IN_TREINEIRO). Neste desafio, você deve criar um modelo de classificação binária para inferir a mesma. Os resultados possíveis da sua resposta devem ser “0” ou “1”.

Salve sua resposta em um arquivo chamado answer.csv com duas colunas: `NU_INSCRICAO` e `IN_TREINEIRO`.

### Tópicos

Neste desafio você aprenderá:
- Python
- Pandas
- Sklearn
- Regression
- Classification

## Setup geral

In [41]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression

## Análise dos dados

Lendo os arquivos de treino (train) e de teste (test).

In [42]:
df_train = pd.read_csv('train.csv')
df_train.head()

,Unnamed: 0,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,...,Q041,Q042,Q043,Q044,Q045,Q046,Q047,Q048,Q049,Q050
0,1,ed50e8aaa58e7a806c337585efee9ca41f1eb1ad,2016,4314902,Porto Alegre,43,RS,24,M,0.0,...,5.0,A,A,A,A,A,A,A,B,D
1,2,2c3acac4b33ec2b195d77e7c04a2d75727fad723,2016,2304707,Granja,23,CE,17,F,0.0,...,NaN,A,A,C,A,B,A,A,C,A
2,3,f4545f8ccb9ff5c8aad7d32951b3f251a26e6568,2016,2304400,Fortaleza,23,CE,21,F,0.0,...,NaN,A,A,A,A,C,A,A,B,A
3,4,3d6ec248fef899c414e77f82d5c6d2bffbeaf7fe,2016,3304557,Rio de Janeiro,33,RJ,25,F,0.0,...,5.0,C,A,A,A,A,D,A,A,A
4,5,bf896ac8d3ecadd6dba1dfbf50110afcbf5d3268,2016,1302603,Manaus,13,AM,28,M,0.0,...,NaN,A,A,A,A,A,A,A,A,A


In [43]:
df_train.columns

Index(['Unnamed: 0', 'NU_INSCRICAO', 'NU_ANO', 'CO_MUNICIPIO_RESIDENCIA',
       'NO_MUNICIPIO_RESIDENCIA', 'CO_UF_RESIDENCIA', 'SG_UF_RESIDENCIA',
       'NU_IDADE', 'TP_SEXO', 'TP_ESTADO_CIVIL',
       ...
       'Q041', 'Q042', 'Q043', 'Q044', 'Q045', 'Q046', 'Q047', 'Q048', 'Q049',
       'Q050'],
      dtype='object', length=167)

Dropando a coluna 'Unnamed: 0'.

In [44]:
df_train.drop('Unnamed: 0', axis=1, inplace=True)
df_train.head()

,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,...,Q041,Q042,Q043,Q044,Q045,Q046,Q047,Q048,Q049,Q050
0,ed50e8aaa58e7a806c337585efee9ca41f1eb1ad,2016,4314902,Porto Alegre,43,RS,24,M,0.0,1,...,5.0,A,A,A,A,A,A,A,B,D
1,2c3acac4b33ec2b195d77e7c04a2d75727fad723,2016,2304707,Granja,23,CE,17,F,0.0,3,...,NaN,A,A,C,A,B,A,A,C,A
2,f4545f8ccb9ff5c8aad7d32951b3f251a26e6568,2016,2304400,Fortaleza,23,CE,21,F,0.0,3,...,NaN,A,A,A,A,C,A,A,B,A
3,3d6ec248fef899c414e77f82d5c6d2bffbeaf7fe,2016,3304557,Rio de Janeiro,33,RJ,25,F,0.0,0,...,5.0,C,A,A,A,A,D,A,A,A
4,bf896ac8d3ecadd6dba1dfbf50110afcbf5d3268,2016,1302603,Manaus,13,AM,28,M,0.0,2,...,NaN,A,A,A,A,A,A,A,A,A


In [45]:
df_test = pd.read_csv('test.csv')
df_test.head()

,NU_INSCRICAO,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,...,NU_NOTA_COMP5,NU_NOTA_REDACAO,Q001,Q002,Q006,Q024,Q025,Q026,Q027,Q047
0,ba0cc30ba34e7a46764c09dfc38ed83d15828897,43,RS,19,F,1,1,1,3,1,...,NaN,NaN,E,E,H,B,B,C,D,A
1,177f281c68fa032aedbd842a745da68490926cd2,15,PA,24,M,3,2,1,4,1,...,NaN,NaN,B,C,B,A,A,C,D,A
2,6cf0d8b97597d7625cdedc7bdb6c0f052286c334,29,BA,16,F,2,1,3,0,1,...,NaN,NaN,E,F,G,B,B,A,NaN,D
3,5c356d810fa57671402502cd0933e5601a2ebf1e,41,PR,17,F,1,1,2,0,2,...,40.0,480.0,E,E,E,C,B,B,C,A
4,df47c07bd881c2db3f38c6048bf77c132ad0ceb3,53,DF,19,F,1,1,1,1,1,...,80.0,720.0,E,E,E,B,B,B,D,A


Antes de manipular os dataframes, deve-separar as colunas de notas da prova de matemática do treino e a do número de inscrição do teste.

In [46]:
train_y = df_train['IN_TREINEIRO'].fillna(0)

In [47]:
nu_inscricao = df_test['NU_INSCRICAO']

Idealmente os arquivos de teste e treino teriam as mesmas colunas, exceto a que deve ser predita. Então, primeiro verifica-se a quantidade de colunas de cada, e depois exclui-se as colunas que não pertence a ambas.

In [48]:
colunas_intersecao = np.intersect1d(df_train.columns.values, df_test.columns.values)
colunas_intersecao, len(colunas_intersecao)

(array(['CO_UF_RESIDENCIA', 'IN_BAIXA_VISAO', 'IN_CEGUEIRA',
        'IN_DISCALCULIA', 'IN_DISLEXIA', 'IN_GESTANTE', 'IN_IDOSO',
        'IN_SABATISTA', 'IN_SURDEZ', 'NU_IDADE', 'NU_INSCRICAO',
        'NU_NOTA_CH', 'NU_NOTA_CN', 'NU_NOTA_COMP1', 'NU_NOTA_COMP2',
        'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5', 'NU_NOTA_LC',
        'NU_NOTA_REDACAO', 'Q001', 'Q002', 'Q006', 'Q024', 'Q025', 'Q026',
        'Q027', 'Q047', 'SG_UF_RESIDENCIA', 'TP_ANO_CONCLUIU',
        'TP_COR_RACA', 'TP_DEPENDENCIA_ADM_ESC', 'TP_ENSINO', 'TP_ESCOLA',
        'TP_LINGUA', 'TP_NACIONALIDADE', 'TP_PRESENCA_CH',
        'TP_PRESENCA_CN', 'TP_PRESENCA_LC', 'TP_PRESENCA_MT', 'TP_SEXO',
        'TP_STATUS_REDACAO', 'TP_ST_CONCLUSAO'], dtype=object),
 43)

In [49]:
df_train = df_train[colunas_intersecao]
df_train.head()

,CO_UF_RESIDENCIA,IN_BAIXA_VISAO,IN_CEGUEIRA,IN_DISCALCULIA,IN_DISLEXIA,IN_GESTANTE,IN_IDOSO,IN_SABATISTA,IN_SURDEZ,NU_IDADE,...,TP_ESCOLA,TP_LINGUA,TP_NACIONALIDADE,TP_PRESENCA_CH,TP_PRESENCA_CN,TP_PRESENCA_LC,TP_PRESENCA_MT,TP_SEXO,TP_STATUS_REDACAO,TP_ST_CONCLUSAO
0,43,0,0,0,0,0,0,0,0,24,...,1,1,1,1,1,1,1,M,1.0,1
1,23,0,0,0,0,0,0,0,0,17,...,2,1,1,1,1,1,1,F,1.0,2
2,23,0,0,0,0,0,0,0,0,21,...,1,1,1,0,0,0,0,F,NaN,3
3,33,0,0,0,0,0,0,0,0,25,...,1,0,1,0,0,0,0,F,NaN,1
4,13,0,0,0,0,0,0,0,0,28,...,1,1,1,0,0,0,0,M,NaN,1


In [50]:
df_test = df_test[colunas_intersecao]
df_test.head()

,CO_UF_RESIDENCIA,IN_BAIXA_VISAO,IN_CEGUEIRA,IN_DISCALCULIA,IN_DISLEXIA,IN_GESTANTE,IN_IDOSO,IN_SABATISTA,IN_SURDEZ,NU_IDADE,...,TP_ESCOLA,TP_LINGUA,TP_NACIONALIDADE,TP_PRESENCA_CH,TP_PRESENCA_CN,TP_PRESENCA_LC,TP_PRESENCA_MT,TP_SEXO,TP_STATUS_REDACAO,TP_ST_CONCLUSAO
0,43,0,0,0,0,0,0,0,0,19,...,1,1,1,1,1,0,0,F,NaN,1
1,15,0,0,0,0,0,0,0,0,24,...,1,1,2,0,0,0,0,M,NaN,1
2,29,0,0,0,0,0,0,0,0,16,...,1,0,1,1,1,0,0,F,NaN,3
3,41,0,0,0,0,0,0,0,0,17,...,2,1,1,1,1,1,1,F,1.0,2
4,53,0,0,0,0,0,0,0,0,19,...,1,1,1,1,1,1,1,F,1.0,1


In [51]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13730 entries, 0 to 13729
Data columns (total 43 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   CO_UF_RESIDENCIA        13730 non-null  int64  
 1   IN_BAIXA_VISAO          13730 non-null  int64  
 2   IN_CEGUEIRA             13730 non-null  int64  
 3   IN_DISCALCULIA          13730 non-null  int64  
 4   IN_DISLEXIA             13730 non-null  int64  
 5   IN_GESTANTE             13730 non-null  int64  
 6   IN_IDOSO                13730 non-null  int64  
 7   IN_SABATISTA            13730 non-null  int64  
 8   IN_SURDEZ               13730 non-null  int64  
 9   NU_IDADE                13730 non-null  int64  
 10  NU_INSCRICAO            13730 non-null  object 
 11  NU_NOTA_CH              10341 non-null  float64
 12  NU_NOTA_CN              10341 non-null  float64
 13  NU_NOTA_COMP1           10133 non-null  float64
 14  NU_NOTA_COMP2           10133 non-null

Apenas as colunas numéricas foram usadas na predição.

In [52]:
colunas_numericas = df_train.select_dtypes(include=['float64', 'int64']).columns
colunas_numericas

Index(['CO_UF_RESIDENCIA', 'IN_BAIXA_VISAO', 'IN_CEGUEIRA', 'IN_DISCALCULIA',
       'IN_DISLEXIA', 'IN_GESTANTE', 'IN_IDOSO', 'IN_SABATISTA', 'IN_SURDEZ',
       'NU_IDADE', 'NU_NOTA_CH', 'NU_NOTA_CN', 'NU_NOTA_COMP1',
       'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5',
       'NU_NOTA_LC', 'NU_NOTA_REDACAO', 'TP_ANO_CONCLUIU', 'TP_COR_RACA',
       'TP_DEPENDENCIA_ADM_ESC', 'TP_ENSINO', 'TP_ESCOLA', 'TP_LINGUA',
       'TP_NACIONALIDADE', 'TP_PRESENCA_CH', 'TP_PRESENCA_CN',
       'TP_PRESENCA_LC', 'TP_PRESENCA_MT', 'TP_STATUS_REDACAO',
       'TP_ST_CONCLUSAO'],
      dtype='object')

In [53]:
df_train = df_train[colunas_numericas].fillna(0)
df_train.head()

,CO_UF_RESIDENCIA,IN_BAIXA_VISAO,IN_CEGUEIRA,IN_DISCALCULIA,IN_DISLEXIA,IN_GESTANTE,IN_IDOSO,IN_SABATISTA,IN_SURDEZ,NU_IDADE,...,TP_ENSINO,TP_ESCOLA,TP_LINGUA,TP_NACIONALIDADE,TP_PRESENCA_CH,TP_PRESENCA_CN,TP_PRESENCA_LC,TP_PRESENCA_MT,TP_STATUS_REDACAO,TP_ST_CONCLUSAO
0,43,0,0,0,0,0,0,0,0,24,...,0.0,1,1,1,1,1,1,1,1.0,1
1,23,0,0,0,0,0,0,0,0,17,...,1.0,2,1,1,1,1,1,1,1.0,2
2,23,0,0,0,0,0,0,0,0,21,...,0.0,1,1,1,0,0,0,0,0.0,3
3,33,0,0,0,0,0,0,0,0,25,...,0.0,1,0,1,0,0,0,0,0.0,1
4,13,0,0,0,0,0,0,0,0,28,...,0.0,1,1,1,0,0,0,0,0.0,1


In [54]:
df_test = df_test[colunas_numericas].fillna(0)
df_test.head()

,CO_UF_RESIDENCIA,IN_BAIXA_VISAO,IN_CEGUEIRA,IN_DISCALCULIA,IN_DISLEXIA,IN_GESTANTE,IN_IDOSO,IN_SABATISTA,IN_SURDEZ,NU_IDADE,...,TP_ENSINO,TP_ESCOLA,TP_LINGUA,TP_NACIONALIDADE,TP_PRESENCA_CH,TP_PRESENCA_CN,TP_PRESENCA_LC,TP_PRESENCA_MT,TP_STATUS_REDACAO,TP_ST_CONCLUSAO
0,43,0,0,0,0,0,0,0,0,19,...,0.0,1,1,1,1,1,0,0,0.0,1
1,15,0,0,0,0,0,0,0,0,24,...,0.0,1,1,2,0,0,0,0,0.0,1
2,29,0,0,0,0,0,0,0,0,16,...,0.0,1,0,1,1,1,0,0,0.0,3
3,41,0,0,0,0,0,0,0,0,17,...,1.0,2,1,1,1,1,1,1,1.0,2
4,53,0,0,0,0,0,0,0,0,19,...,0.0,1,1,1,1,1,1,1,1.0,1


In [55]:
reg = LogisticRegression()
reg.fit(df_train, train_y)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [56]:
y_teste = reg.predict(df_test)

Depois de predizer se foi por treino ou não, usando o arquivo de teste, estas foram salvar em um arquivo csv.

In [57]:
answer = pd.DataFrame()
answer['NU_INSCRICAO'] = nu_inscricao
answer['IN_TREINEIRO'] = y_teste
answer.head()

,NU_INSCRICAO,IN_TREINEIRO
0,ba0cc30ba34e7a46764c09dfc38ed83d15828897,0
1,177f281c68fa032aedbd842a745da68490926cd2,0
2,6cf0d8b97597d7625cdedc7bdb6c0f052286c334,1
3,5c356d810fa57671402502cd0933e5601a2ebf1e,0
4,df47c07bd881c2db3f38c6048bf77c132ad0ceb3,0


In [58]:
answer.to_csv('answer.csv', index=False)